## 라이브러리

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import numpy as np

## 예제 문장 정의 및 토큰화

In [6]:
# 문장 정의
sentence = 'I want to study NLP using RNN.'

In [7]:
# 토큰화
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([sentence])

## 입력 데이터 전처리

In [8]:
# 문자를 정수 인덱스로 변환
encoded = tokenizer.texts_to_sequences([sentence])[0]

In [9]:
# 입력 시퀀스와 출력 시퀀스 준비
sequences = []
for i in range(1, len(encoded)):
    sequence = encoded[i-1:i+1]
    sequences.append(sequence)

sequences = np.array(sequences)

In [10]:
# 입력(X)과 출력(y) 분리
X, y = sequences[:,0],sequences[:,1]

In [11]:
# 출력 데이터를 one-hot 인코딩
y = to_categorical(y, num_classes=len(tokenizer.word_index)+1)

In [12]:
# 입력 데이터를 3D 텐서로 변환
X = np.reshape(X, (X.shape[0], 1, 1))

## 모델 생성

In [14]:
model = Sequential()
model.add(SimpleRNN(50, input_shape=(1, 1), return_sequences=False))
model.add(Dense(len(tokenizer.word_index)+1, activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 요약
model.summary()


# 모델 학습 (여기서는 예제로 3 에포크만 실행)
model.fit(X, y, epochs=3, verbose=2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 50)                2600      
                                                                 
 dense_1 (Dense)             (None, 17)                867       
                                                                 
Total params: 3467 (13.54 KB)
Trainable params: 3467 (13.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/3
1/1 - 1s - loss: 3.0685 - accuracy: 0.0345 - 1s/epoch - 1s/step
Epoch 2/3
1/1 - 0s - loss: 3.0404 - accuracy: 0.0345 - 7ms/epoch - 7ms/step
Epoch 3/3
1/1 - 0s - loss: 3.0130 - accuracy: 0.0345 - 7ms/epoch - 7ms/step


## 문자를 주어진 다음 문자로 매핑하는 함수

In [15]:
def predict_next_char(model, tokenizer, char_to_predict):
    # 주어진 문자를 정수로 변환
    char_index = tokenizer.texts_to_sequences([char_to_predict])[0]
    char_index = np.array(char_index).reshape(1, 1, 1)

    # 예측
    prediction = model.predict(char_index, verbose=0)

    # 가장 높은 확률을 가진 정수 인덱스를 찾기
    predicted_char_index = np.argmax(prediction)

    # 정수를 문자로 다시 변환
    predicted_char = tokenizer.sequences_to_texts([[predicted_char_index]])[0]
    return predicted_char

## 예측 테스트

In [21]:
input_char = ''
predicted_char = predict_next_char(model, tokenizer, input_char)
print('Given character:', input_char)
print('Next character prediction:', predicted_char)

Given character: t
Next character prediction: r
